# `dPdI.ipynb`

Calculate dPdI from dIdV output, use to calculate NEP from PSD

In [ ]:
# imports
import sys
import os
import pickle
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as ss
import scipy.optimize as so
import scipy.integrate as si
import scipy.stats as st


# custom modules
from picoscopedata import PicoscopeData
from utilities import *


find_imports()
import qetpy as qp
import optimal_filter_utils as of


In [ ]:
# base location of "B13_Runs" folder
folder_base_candidates = [
    'D:', # on the cryolab2 computer
    '/Volumes/B13Cryolab_Share' # connecting to shared drives from Mac
]

folder_base = find_folder('B13_Runs', *folder_base_candidates)
print(folder_base)

In [19]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

required data inputs

In [33]:
# read in results from IbIs analysis
#with open('./ibis_data/local/20241108-0001_results.dat', 'rb') as f:
#    ibis, ivfits, tr_fits = pickle.load(f)

with open('./didv_data/local/20241108-0002_results.dat', 'rb') as f:
    didvfit = pickle.load(f)

channels = 'ABCDE'

# needed for power spectrum of pulses
psd_path = folder_base + '/B13_Runs_LabVIEW/Run94/Sapphire_ChABCD_SWT_weekend/Analysis 13/'


# needed for phonon collection efficiency calc

# position of peaks in calibration (OF height where peak occurs)
peak_positions = {
    'A': None,
    'B': 6.47901e-8,#1.78448e-7,
    'C': 5.42406e-8, #8.97955e-8,
    'D': None,
    'E': None
}

peak_energy = 6. # energy of peak

In [ ]:
# calculate NEP from PSD data and write to txt files

psd_data_output_dir = folder_base + 'B13_Runs_LabVIEW/Run94/'

def read_PSD(filename, nsamples, trace_length):
    fs = nsamples/trace_length
    psd = np.loadtxt(filename)
    fs, _ = qp.utils.fold_spectrum(qp.utils.fft(np.zeros(2*psd.size-2), fs = fs)[0], fs = fs)
    return fs, psd

fig1, ax1 = plt.subplots(1,1)
ax1.set_title('PSDs')

fig2, ax2 = plt.subplots(1,1)
ax2.set_title('NEP')


dpdi = {}

for chan in channels:
    fs, psd = read_PSD(psd_path + f'PSD/Ch {chan} PSD.txt', nsamples = 1000_000, trace_length = 1)
    dpdi[chan], dpdi_err = qp.get_dPdI_with_uncertainties(fs, didvfit[chan][3])

    #np.savetxt(psd_data_output_dir + f'Ch{chan}.txt', abs(dpdi[chan]))

    ax1.plot(fs, psd, label = f'Ch {chan}')
    ax2.plot(fs, 1e15*psd*abs(dpdi[chan]), label = f'Ch {chan}')


ax1.grid()
ax2.grid()
ax1.loglog()
ax2.loglog()
ax1.set_xlabel('freq (Hz)')
ax2.set_xlabel('freq (Hz)')
ax1.set_ylabel('A/sqrt(Hz)')
ax2.set_ylabel('fW/sqrt(Hz)')
ax1.legend()
ax2.legend()



In [ ]:
# compare to theoretical NEP spectrum from mpyle Ch 3

with open('Sp.txt', 'rb') as f:
    Sp, fn = pickle.load(f)


fig2, ax2 = plt.subplots(1,1)
ax2.set_title('NEP')

for chan in channels:
    fs, psd = read_PSD(psd_path + f'PSD/Ch {chan} PSD.txt', nsamples = 1000_000, trace_length = 1)

    ax2.plot(fs, 1e15*psd*abs(dpdi[chan]), label = f'Ch {chan}')

ax2.plot(fn, 1e15*np.sqrt(Sp), label = 'theoretical', color = 'k')


ax2.grid() 
ax2.loglog()
ax2.set_xlabel('freq (Hz)')
ax2.set_ylabel('fW/sqrt(Hz)')
ax2.legend()



In [ ]:
dpdi['A'].shape, psd.shape

In [24]:
# np.savetxt(psd_data_output_dir + f'Ch{chan}.txt', abs(dpdi[chan]))

In [25]:
# get pulse templates

templates_unnorm = {}
templates = {}

for c in channels:

    templates_unnorm[c] = np.loadtxt(psd_path + f'Templates 2/Ch {c} Template.txt')

    templates[c] = templates_unnorm[c]/templates_unnorm[c].max()

_, ts_template, ws_template = of.get_idx_tk_wn(templates['A'].size, 1e-3) # templates = 2 ms traces

fs_template = ws_template/2/np.pi # frequencies for template frequency space [Hz]

In [ ]:
plt.figure()

for c in channels:
    plt.plot(ts_template*1e3, templates[c], label = f'Ch {c}')

#plt.axis((-0.2, 0.2, 0.5,1.1))
plt.title('pulse templates')
plt.grid()
plt.legend()
plt.xlabel('time (ms)')

In [ ]:
plt.figure()

for c in channels:
    plt.plot(abs(of.fft(templates[c])[:templates[c].size//2]), label = f'Ch {c}')

plt.loglog()
#plt.axis((-0.2, 0.2, 0.5,1.1))
plt.title('pulse templates')
plt.grid()

In [28]:
# convert templates to frequency domain and convert to power
pulse_power = {}

template_energy = {} # energy (keV) of template

edge_cut = 5

for c in channels:

    temp_w = of.fft(templates[c])

    #dpdi_unfolded = np.hstack((dpdi[c][::-1], dpdi[c]))/2
    dpdi_unfolded = np.hstack((abs(dpdi[c]), abs(dpdi[c])[::-1]))/2

    pow_w = temp_w*dpdi_unfolded

    pulse_power[c] = of.ifft(pow_w)

    template_energy[c] = np.trapz(np.real(pulse_power[c])[edge_cut:-edge_cut], ts_template[edge_cut:-edge_cut]) / 1.602e-16
    

In [ ]:
plt.figure()

for c in channels:
    plt.plot(ts_template[edge_cut:-edge_cut]*1e3, np.real(pulse_power[c][edge_cut:-edge_cut]), label = f'Ch {c}')

plt.title('pulse power')
plt.grid()
plt.legend()
plt.ylabel('power [W]')
plt.xlabel('time (ms)')

In [ ]:
template_energy

In [ ]:
# calculate phonon collection efficiency

print('phonon collection efficiencies')
for c in channels:
    if peak_positions[c]:
        unit_energy = 6./peak_positions[c] # energy of pulse with unit height
        print(c, ':', f'{100*template_energy[c]/unit_energy:.5g} %')

In [ ]:
# cumulative energy
plt.figure()

for c in channels:
    plt.plot(1e3*ts_template[1:], si.cumulative_trapezoid(np.real(pulse_power[c])/1.6e-16, ts_template), label = f'Ch {c}')

plt.title('integral of pulse power')
plt.grid()
plt.legend()
plt.ylabel('energy [keV]')
plt.xlabel('time (ms)')